In [1]:
import sys # for automation and parallelization: set manual to false when run by a launcher
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

# START

In [2]:
import sys
sys.path.insert(0, r'../../../quetzal')

from quetzal.model import stepmodel
import numpy as np
import pandas as pd
from quetzal.io import excel

SQLalchemy is not installed. No support for SQL output.


In [3]:
training_folder = '../../'

In [4]:
# the scenario specific variables are read from the parameter file
var = excel.read_var(file='../../inputs/parameters.xlsx', scenario=scenario)

In [5]:
sm = stepmodel.read_zippedpickles(training_folder + 'model/{scen}/aon_pathfinder'.format(scen=scenario))

zone_to_transit: 100%|█████████████████████████████████████████████████████████████████| 38/38 [00:01<00:00, 21.16it/s]


# generation
- Production = Population
- Attraction = Jobs

In [6]:
#Linear regressions coefficients for production and attraction estimation

prod = {'work' : [0.0988, 0, 0],
       'back_home' : [0, 0.0493, 0],
       'others' : [0.2246, 0, 0]}

attr = {'work' : [0, 0.2518, 0],
       'back_home' : [0.0202, 0, 0],
       'others' : [0, 0, 0.3609]}

prod = pd.DataFrame(prod, index = ['population', 'empl_estim', 'visit_estim'])
attr = pd.DataFrame(attr, index = ['population', 'empl_estim', 'visit_estim'])
prod.index.name = 'variable'
attr.index.name = 'variable'
prod.columns.name = 'purpose'
attr.columns.name = 'purpose'

df = pd.DataFrame({'prod':prod.stack(), 'attr':attr.stack()})

df.columns.name = 'direction'

coef = df.unstack('purpose')

coef *= 0.885

coef

direction        prod                          attr                    
purpose          work back_home    others      work back_home    others
variable                                                               
population   0.087438   0.00000  0.198771  0.000000  0.017877  0.000000
empl_estim   0.000000   0.04363  0.000000  0.222843  0.000000  0.000000
visit_estim  0.000000   0.00000  0.000000  0.000000  0.000000  0.319397

In [7]:
#Purpose aggregation

coef_agg = coef.stack(['purpose'])
coef_agg = coef_agg.groupby(['variable']).sum()

coef_agg

direction,prod,attr
variable,,
empl_estim,0.043630,0.222843
population,0.286209,0.017877
visit_estim,0.000000,0.319397


In [8]:
shape = str(var['general', 'urban_shape'])

In [9]:
if var['jobs']['nip']!=0:
    sm.zones['Jobs ' + shape]['zone_194']=var['jobs']['nip']

In [10]:
sm.zones['population'] = sm.zones['Population ' + shape]
sm.zones['empl_estim'] = sm.zones['Jobs ' + shape]
sm.zones['visit_estim'] = sm.zones['Visits ' + shape]
zones_var = sm.zones.copy()
zones_var = zones_var[['population', 'empl_estim', 'visit_estim']]

In [11]:
sm.zones['population'].sum()

319476.0

In [12]:
sm.zones['empl_estim'].sum()

106912.0

In [13]:
coef_agg

direction,prod,attr
variable,,
empl_estim,0.043630,0.222843
population,0.286209,0.017877
visit_estim,0.000000,0.319397


In [14]:
zones_survey = pd.read_excel(r'../../../quetzal_dire_dawa/inputs/production_attraction_zones.xlsx')
zones_survey['latin_name']=zones_survey['latin_name'].apply(lambda n: n.replace(" ","_"))
zones_survey['prod_hpm']=zones_survey['prod_hpm_work']+zones_survey['prod_hpm_back_home']+zones_survey['prod_hpm_others']
zones_survey['attr_hpm']=zones_survey['attr_hpm_work']+zones_survey['attr_hpm_back_home']+zones_survey['attr_hpm_others']

In [15]:
#Generation

if var['step_distribution']['generation_from']=='model':
    sm.zones['emission']=zones_var.dot(coef_agg['prod'])
    sm.zones['attraction']=zones_var.dot(coef_agg['attr'])
elif var['step_distribution']['generation_from']=='counts':
    zones_counts = sm.zones.reset_index()
    zones_counts = pd.merge(zones_counts,zones_survey[['latin_name','prod_hpm','attr_hpm']],
                            how='left',left_on='latin_name',right_on='latin_name')
    zones_counts['prod_hpm']=zones_counts['prod_hpm'].fillna(0)
    zones_counts['attr_hpm']=zones_counts['attr_hpm'].fillna(0)
    sm.zones['emission']=zones_counts.set_index('id')['prod_hpm']
    sm.zones['attraction']=zones_counts.set_index('id')['attr_hpm']

In [19]:
sm.zones['attraction'].sum()

87412.42544700002

# car owners
car owners distribution is based on car times

In [ ]:
motor_rate = 0
car = sm.copy()
car.zones['emission'] *= motor_rate

In [ ]:
imp_matrix = car.car_los[
    ['origin', 'destination', 'time']
].set_index(['origin','destination']).unstack() 
imp_matrix = imp_matrix.replace(0, 0.1)

car.step_distribution(deterrence_matrix=imp_matrix)

In [ ]:
dict_area=sm.zones[['area']].to_dict('index') #TODO check 

# PT captives
car owners distribution is based on car times

In [ ]:
pt = sm.copy()
pt.zones['emission'] *= (1-motor_rate)

#pt.pt_los['gtime']=np.clip(pt.pt_los['gtime'], 900, None)

# This replace np.clip -> Intrazonal time depends on the zone area

pt.pt_los['area_origin']=pt.pt_los['origin'].apply(lambda i: dict_area[i]['area'])
pt.pt_los

coef = var['step_distribution']['pt_intrazonal_parameter'] #To calibrate
walk_speed = var['speed']['walk_on_road'] #km/h

import math

for element in pt.pt_los.index:
    if pt.pt_los['origin'][element]==pt.pt_los['destination'][element]:
        pt.pt_los['gtime'][element]=3600*coef*math.sqrt(pt.pt_los['area_origin'][element])/walk_speed

# Add X seconds to each OD

pt.pt_los['gtime'] = pt.pt_los['gtime'] + var['step_distribution']['pt_intrazonal_time']

# Ici on prend comme fonction de coût le temps TC. Notez le .unstack() 

imp_matrix = pt.pt_los[
    ['origin', 'destination', 'gtime']
].set_index(['origin','destination']).unstack() 

#imp_matrix = imp_matrix.replace(0, 120)

In [ ]:
imp_matrix.head()

In [ ]:
sm.zones.head()

In [ ]:
pt.pt_los

In [ ]:
import math

imp_matrix = pt.pt_los[
    ['origin', 'destination', 'gtime']
].set_index(['origin','destination'])
#.unstack()

alpha = var['step_distribution']['pt_power'] #1.5-2.5
beta = var['step_distribution']['pt_exponential_weight'] #0.25 hours

#Combined Impedance Fonction f(tij) = tij^(alpha) * exp(beta*tij)
imp_matrix['gtime']=imp_matrix['gtime'].apply(lambda s: (pow(s/3600,alpha)*math.exp(beta*s/3600)))

imp_matrix = imp_matrix.unstack()

serie=pd.Series(range(120,3600,60))
df=pd.DataFrame(serie)
df['p']=serie.apply(lambda s: (pow(s/3600,alpha)*math.exp(beta*s/3600)))
df['p']=df['p']/df['p'].max()
df.set_index(0).plot()

In [ ]:
order = pt.zones.index
imp_matrix = imp_matrix['gtime'].loc[order, order]

In [ ]:
pt.step_distribution(deterrence_matrix=imp_matrix)

In [ ]:
pt.volumes.head()

# merge matrices

In [ ]:
sm.volumes = pd.merge(
    car.volumes,
    pt.volumes,
    on=['origin', 'destination'],
    suffixes=[ '_car_owner', '_pt_captive']
).rename(columns={'volume_car_owner': 'car_owner', 'volume_pt_captive': 'pt_captive'})

# to_zip

In [ ]:
sm.to_zippedpickles(
    training_folder + 'model/{scen}/distribution'.format(scen=scenario), 
    only_attributes=['volumes', 'zones','epsg', 'coordinates_unit']
)

In [ ]:
end_of_notebook